# Práce s modely Mistral

## Úvod

V této lekci se dozvíte:
- Jaké různé modely Mistral existují
- Kdy a v jakých situacích je vhodné jednotlivé modely použít
- Ukázky kódu, které představují jedinečné vlastnosti každého modelu


## Mistral modely

V této lekci si představíme 3 různé modely Mistral:  
**Mistral Large**, **Mistral Small** a **Mistral Nemo**.

Každý z těchto modelů je zdarma dostupný na tržišti Github Model. Kód v tomto notebooku bude tyto modely využívat pro spouštění kódu. Další informace o využití Github Models k [prototypování s AI modely](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst) najdete zde.


## Mistral Large 2 (2407)
Mistral Large 2 je v současnosti vlajkovým modelem společnosti Mistral a je určený pro firemní použití.

Model je vylepšením původního Mistral Large a nabízí:
-  Větší kontextové okno – 128k oproti 32k
-  Lepší výkon v matematických a programovacích úlohách – průměrná přesnost 76,9 % oproti 60,4 %
-  Vyšší vícejazyčný výkon – podporované jazyky zahrnují: angličtinu, francouzštinu, němčinu, španělštinu, italštinu, portugalštinu, nizozemštinu, ruštinu, čínštinu, japonštinu, korejštinu, arabštinu a hindštinu.

Díky těmto vlastnostem vyniká Mistral Large v:
- *Retrieval Augmented Generation (RAG)* – díky většímu kontextovému oknu
- *Function Calling* – tento model má nativní podporu volání funkcí, což umožňuje integraci s externími nástroji a API. Tyto volání lze provádět jak paralelně, tak i postupně jedno po druhém.
- *Generování kódu* – model je špičkový při generování kódu v jazycích Python, Java, TypeScript a C++.


V tomto příkladu používáme Mistral Large 2 k provedení RAG vzoru nad textovým dokumentem. Otázka je napsaná v korejštině a ptá se na autorovy aktivity před vysokou školou.

Používá se model Cohere Embeddings k vytvoření embeddingů jak textového dokumentu, tak otázky. Pro tento příklad je jako vektorový úložiště použit Python balíček faiss.

Prompt, který je odeslán do modelu Mistral, obsahuje jak otázku, tak nalezené úryvky, které jsou otázce podobné. Model pak poskytne odpověď v přirozeném jazyce.


In [50]:
pip install faiss-cpu

Note: you may need to restart the kernel to use updated packages.


In [51]:
import requests
import numpy as np
import faiss
import os

from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from azure.ai.inference import EmbeddingsClient

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = requests.get('https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt')
text = response.text

chunk_size = 2048
chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
len(chunks)

embed_model_name = "cohere-embed-v3-multilingual" 

embed_client = EmbeddingsClient(
        endpoint=endpoint,
        credential=AzureKeyCredential(token)
)

embed_response = embed_client.embed(
    input=chunks,
    model=embed_model_name
)



text_embeddings = []
for item in embed_response.data:
    length = len(item.embedding)
    text_embeddings.append(item.embedding)
text_embeddings = np.array(text_embeddings)


d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

question = "저자가 대학에 오기 전에 주로 했던 두 가지 일은 무엇이었나요?？"

question_embedding = embed_client.embed(
    input=[question],
    model=embed_model_name
)

question_embeddings = np.array(question_embedding.data[0].embedding)


D, I = index.search(question_embeddings.reshape(1, -1), k=2) # distance, index
retrieved_chunks = [chunks[i] for i in I.tolist()[0]]

prompt = f"""
Context information is below.
---------------------
{retrieved_chunks}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {question}
Answer:
"""


chat_response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content=prompt),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(chat_response.choices[0].message.content)

The author primarily engaged in two activities before college: writing and programming. In terms of writing, they wrote short stories, albeit not very good ones, with minimal plot and characters expressing strong feelings. For programming, they started writing programs on the IBM 1401 used for data processing during their 9th grade, at the age of 13 or 14. They used an early version of Fortran and typed programs on punch cards, later loading them into the card reader to run the program.


## Mistral Small 
Mistral Small je další model z rodiny Mistral, který spadá do kategorie premier/enterprise. Jak už název napovídá, jedná se o malý jazykový model (SLM). Výhody použití Mistral Small jsou:
- Úspora nákladů ve srovnání s Mistral LLM, jako je Mistral Large a NeMo – snížení ceny o 80 %
- Nízká latence – rychlejší odezva oproti LLM modelům Mistral
- Flexibilita – lze nasadit v různých prostředích s menšími nároky na potřebné zdroje.

Mistral Small je ideální pro:
- Úlohy založené na textu, jako je sumarizace, analýza sentimentu a překlad.
- Aplikace, kde je potřeba často odesílat požadavky díky jeho cenové efektivitě
- Úkoly s nízkou latencí, například kontrola kódu a návrhy kódu


## Porovnání Mistral Small a Mistral Large

Pro zobrazení rozdílů v latenci mezi Mistral Small a Large spusťte níže uvedené buňky.

Měli byste zaznamenat rozdíl v době odezvy mezi 3-5 sekundami. Všimněte si také rozdílů v délce a stylu odpovědí na stejný prompt.


In [ ]:
import os 
endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-small"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

endpoint = "https://models.inference.ai.azure.com"
model_name = "Mistral-large"
token = os.environ["GITHUB_TOKEN"]

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful coding assistant."),
        UserMessage(content="Can you write a Python function to the fizz buzz test?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)

print(response.choices[0].message.content)

## Mistral NeMo

Ve srovnání s ostatními dvěma modely probíranými v této lekci je Mistral NeMo jediný volně dostupný model s licencí Apache2.

Je považován za vylepšení předchozího open source LLM od Mistral, Mistral 7B.

Další vlastnosti modelu NeMo:

- *Efektivnější tokenizace:* Tento model používá tokenizer Tekken místo běžněji používaného tiktoken. Díky tomu dosahuje lepšího výkonu ve více jazycích i v kódu.

- *Doladění (Finetuning):* Základní model je dostupný pro doladění. To umožňuje větší flexibilitu v případech, kdy je doladění potřeba.

- *Nativní volání funkcí* – Stejně jako Mistral Large byl tento model trénován na volání funkcí. Díky tomu je jedinečný jako jeden z prvních open source modelů, které tuto schopnost mají.


## Mistral NeMo

Ve srovnání s ostatními dvěma modely probíranými v této lekci je Mistral NeMo jediný volně dostupný model s licencí Apache2.

Je považován za vylepšení předchozího open source LLM od Mistral, Mistral 7B.

Další vlastnosti modelu NeMo jsou:

- *Efektivnější tokenizace:* Tento model používá tokenizér Tekken místo běžněji používaného tiktoken. Díky tomu dosahuje lepšího výkonu ve více jazycích i v kódu.

- *Možnost doladění:* Základní model je dostupný pro doladění. To poskytuje větší flexibilitu pro případy, kdy je doladění potřeba.

- *Nativní volání funkcí* – Stejně jako Mistral Large byl tento model trénován na volání funkcí. Díky tomu je jedním z prvních open source modelů, které tuto schopnost mají.


### Porovnání tokenizérů

V tomto příkladu se podíváme, jak Mistral NeMo zpracovává tokenizaci ve srovnání s Mistral Large.

Oba příklady používají stejný prompt, ale měli byste si všimnout, že NeMo vrací méně tokenů než Mistral Large.


In [11]:
pip install mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.0/797.0 kB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "open-mistral-nemo	"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

128


In [13]:
# Import needed packages:
from mistral_common.protocol.instruct.messages import (
    UserMessage,
)
from mistral_common.protocol.instruct.request import ChatCompletionRequest
from mistral_common.protocol.instruct.tool_calls import (
    Function,
    Tool,
)
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer

# Load Mistral tokenizer

model_name = "mistral-large-latest"

tokenizer = MistralTokenizer.from_model(model_name)

# Tokenize a list of messages
tokenized = tokenizer.encode_chat_completion(
    ChatCompletionRequest(
        tools=[
            Tool(
                function=Function(
                    name="get_current_weather",
                    description="Get the current weather",
                    parameters={
                        "type": "object",
                        "properties": {
                            "location": {
                                "type": "string",
                                "description": "The city and state, e.g. San Francisco, CA",
                            },
                            "format": {
                                "type": "string",
                                "enum": ["celsius", "fahrenheit"],
                                "description": "The temperature unit to use. Infer this from the users location.",
                            },
                        },
                        "required": ["location", "format"],
                    },
                )
            )
        ],
        messages=[
            UserMessage(content="What's the weather like today in Paris"),
        ],
        model=model_name,
    )
)
tokens, text = tokenized.tokens, tokenized.text

# Count the number of tokens
print(len(tokens))

135


## Učení zde nekončí, pokračujte na své cestě

Po dokončení této lekce se podívejte na naši [sbírku o Generativní AI](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) a dále rozvíjejte své znalosti v oblasti generativní umělé inteligence!



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí AI překladatelské služby [Co-op Translator](https://github.com/Azure/co-op-translator). Přestože se snažíme o přesnost, mějte prosím na paměti, že automatizované překlady mohou obsahovat chyby nebo nepřesnosti. Za autoritativní zdroj by měl být považován původní dokument v jeho rodném jazyce. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné výklady vzniklé v důsledku použití tohoto překladu.
